In [12]:
import os
import open3d as o3d
import numpy as np
from plyfile import PlyData, PlyElement

input_file  = "/home/ubuntu/scene/EchoScene/assets/scenes/bodleian0025.ply"

# Segment Plane

In [14]:
# Load the point cloud
output_file = os.path.splitext(input_file)[0] + "_segment_plane_aligned.ply"  # Add suffix to the file name

# Read PLY data using PlyData for attribute preservation
ply_data = PlyData.read(input_file)
vertex_data = ply_data['vertex'].data

# Extract attributes
points = np.vstack((vertex_data['x'], vertex_data['y'], vertex_data['z'])).T
normals = np.vstack((vertex_data['nx'], vertex_data['ny'], vertex_data['nz'])).T if 'nx' in vertex_data.dtype.names else np.zeros_like(points)
f_dc = np.column_stack([vertex_data[f'f_dc_{i}'] for i in range(3)]) if 'f_dc_0' in vertex_data.dtype.names else np.zeros((points.shape[0], 3))
f_rest = np.column_stack([vertex_data[f'f_rest_{i}'] for i in range(3)]) if 'f_rest_0' in vertex_data.dtype.names else np.zeros((points.shape[0], 3))
opacity = vertex_data['opacity'] if 'opacity' in vertex_data.dtype.names else np.ones((points.shape[0], 1))
scale = np.column_stack([vertex_data[f'scale_{i}'] for i in range(3)]) if 'scale_0' in vertex_data.dtype.names else np.ones((points.shape[0], 3))
rotation = np.column_stack([vertex_data[f'rot_{i}'] for i in range(4)]) if 'rot_0' in vertex_data.dtype.names else np.zeros((points.shape[0], 4))

# Convert to Open3D point cloud for processing
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
if normals.any():
    pcd.normals = o3d.utility.Vector3dVector(normals)

# Downsample the point cloud
pcd_down = pcd #.voxel_down_sample(voxel_size=0.1)

# Segment the ground using RANSAC
plane_model, inliers = pcd_down.segment_plane(distance_threshold=0.01,
                                              ransac_n=3,
                                              num_iterations=1000)
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

# Extract ground points and outliers
ground_cloud = pcd_down.select_by_index(inliers)
non_ground_cloud = pcd_down.select_by_index(inliers, invert=True)

# Align the ground to the XZ plane
# Calculate the rotation matrix
ground_normal = np.array([a, b, c])
target_normal = np.array([0, 1, 0])  # Target is aligned with Y-axis
rotation_axis = np.cross(ground_normal, target_normal)
rotation_angle = np.arccos(np.dot(ground_normal, target_normal) / (np.linalg.norm(ground_normal) * np.linalg.norm(target_normal)))
rotation_matrix = o3d.geometry.get_rotation_matrix_from_axis_angle(rotation_axis * rotation_angle)

# Apply transformation to the downsampled point cloud
pcd_down.rotate(rotation_matrix, center=(0, 0, 0))

# Extract aligned points and normals
aligned_points = np.asarray(pcd_down.points)
aligned_normals = np.asarray(pcd_down.normals) if pcd_down.has_normals() else np.zeros_like(aligned_points)

# Combine aligned attributes
aligned_data = np.zeros(aligned_points.shape[0], dtype=[
    ('x', 'f4'), ('y', 'f4'), ('z', 'f4'),
    ('nx', 'f4'), ('ny', 'f4'), ('nz', 'f4'),
    ('f_dc_0', 'f4'), ('f_dc_1', 'f4'), ('f_dc_2', 'f4'),
    ('f_rest_0', 'f4'), ('f_rest_1', 'f4'), ('f_rest_2', 'f4'),
    ('opacity', 'f4'),
    ('scale_0', 'f4'), ('scale_1', 'f4'), ('scale_2', 'f4'),
    ('rot_0', 'f4'), ('rot_1', 'f4'), ('rot_2', 'f4'), ('rot_3', 'f4'),
])

aligned_data['x'], aligned_data['y'], aligned_data['z'] = aligned_points[:, 0], aligned_points[:, 1], aligned_points[:, 2]
aligned_data['nx'], aligned_data['ny'], aligned_data['nz'] = aligned_normals[:, 0], aligned_normals[:, 1], aligned_normals[:, 2]
aligned_data['f_dc_0'], aligned_data['f_dc_1'], aligned_data['f_dc_2'] = f_dc[:, 0], f_dc[:, 1], f_dc[:, 2]
aligned_data['f_rest_0'], aligned_data['f_rest_1'], aligned_data['f_rest_2'] = f_rest[:, 0], f_rest[:, 1], f_rest[:, 2]
aligned_data['opacity'] = opacity
aligned_data['scale_0'], aligned_data['scale_1'], aligned_data['scale_2'] = scale[:, 0], scale[:, 1], scale[:, 2]
aligned_data['rot_0'], aligned_data['rot_1'], aligned_data['rot_2'], aligned_data['rot_3'] = rotation[:, 0], rotation[:, 1], rotation[:, 2], rotation[:, 3]

# Save the aligned point cloud
aligned_ply = PlyElement.describe(aligned_data, 'vertex')
PlyData([aligned_ply]).write(output_file)

print(f"Aligned point cloud with attributes saved as: {output_file}")


Plane equation: -0.01x + 0.98y + -0.17z + -0.51 = 0
Aligned point cloud with attributes saved as: /home/ubuntu/scene/EchoScene/assets/scenes/bodleian0025_segment_plane_aligned.ply


In [10]:
import numpy as np
from plyfile import PlyData, PlyElement

# Read PLY data
ply_data = PlyData.read(input_file)
vertex_data = ply_data['vertex'].data

# Extract required attributes
points = np.vstack((vertex_data['x'], vertex_data['y'], vertex_data['z'])).T
normals = np.vstack((vertex_data['nx'], vertex_data['ny'], vertex_data['nz'])).T if 'nx' in vertex_data.dtype.names else np.zeros_like(points)
f_dc = np.column_stack([vertex_data[f'f_dc_{i}'] for i in range(3)]) if 'f_dc_0' in vertex_data.dtype.names else np.zeros((points.shape[0], 3))
f_rest = np.column_stack([vertex_data[f'f_rest_{i}'] for i in range(3)]) if 'f_rest_0' in vertex_data.dtype.names else np.zeros((points.shape[0], 3))
opacity = vertex_data['opacity'] if 'opacity' in vertex_data.dtype.names else np.ones((points.shape[0], 1))
scale = np.column_stack([vertex_data[f'scale_{i}'] for i in range(3)]) if 'scale_0' in vertex_data.dtype.names else np.ones((points.shape[0], 3))
rotation = np.column_stack([vertex_data[f'rot_{i}'] for i in range(4)]) if 'rot_0' in vertex_data.dtype.names else np.zeros((points.shape[0], 4))

# Compute the centroid of the point cloud
centroid = np.mean(points, axis=0)

# Center the point cloud at the origin
centered_points = points - centroid

# Compute the inertia tensor
I = np.zeros((3, 3))
for point in centered_points:
    I[0, 0] += point[1]**2 + point[2]**2
    I[1, 1] += point[0]**2 + point[2]**2
    I[2, 2] += point[0]**2 + point[1]**2
    I[0, 1] -= point[0] * point[1]
    I[0, 2] -= point[0] * point[2]
    I[1, 2] -= point[1] * point[2]
I[1, 0] = I[0, 1]
I[2, 0] = I[0, 2]
I[2, 1] = I[1, 2]

# Compute eigenvalues and eigenvectors
eigenvalues, eigenvectors = np.linalg.eigh(I)

# Sort eigenvalues (smallest to largest)
sorted_indices = np.argsort(eigenvalues)
eigenvectors = eigenvectors[:, sorted_indices]

# Create a rotation matrix to align principal axes
rotation_matrix = eigenvectors.T  # Transpose eigenvectors for alignment

# Apply rotation to point cloud and normals
aligned_points = np.dot(centered_points, rotation_matrix.T)
aligned_normals = np.dot(normals, rotation_matrix.T)

# Combine all attributes
aligned_data = np.zeros(points.shape[0], dtype=[
    ('x', 'f4'), ('y', 'f4'), ('z', 'f4'),
    ('nx', 'f4'), ('ny', 'f4'), ('nz', 'f4'),
    ('f_dc_0', 'f4'), ('f_dc_1', 'f4'), ('f_dc_2', 'f4'),
    ('f_rest_0', 'f4'), ('f_rest_1', 'f4'), ('f_rest_2', 'f4'),
    ('opacity', 'f4'),
    ('scale_0', 'f4'), ('scale_1', 'f4'), ('scale_2', 'f4'),
    ('rot_0', 'f4'), ('rot_1', 'f4'), ('rot_2', 'f4'), ('rot_3', 'f4'),
])

aligned_data['x'], aligned_data['y'], aligned_data['z'] = aligned_points[:, 0], aligned_points[:, 1], aligned_points[:, 2]
aligned_data['nx'], aligned_data['ny'], aligned_data['nz'] = aligned_normals[:, 0], aligned_normals[:, 1], aligned_normals[:, 2]
aligned_data['f_dc_0'], aligned_data['f_dc_1'], aligned_data['f_dc_2'] = f_dc[:, 0], f_dc[:, 1], f_dc[:, 2]
aligned_data['f_rest_0'], aligned_data['f_rest_1'], aligned_data['f_rest_2'] = f_rest[:, 0], f_rest[:, 1], f_rest[:, 2]
aligned_data['opacity'] = opacity
aligned_data['scale_0'], aligned_data['scale_1'], aligned_data['scale_2'] = scale[:, 0], scale[:, 1], scale[:, 2]
aligned_data['rot_0'], aligned_data['rot_1'], aligned_data['rot_2'], aligned_data['rot_3'] = rotation[:, 0], rotation[:, 1], rotation[:, 2], rotation[:, 3]

# Save the aligned point cloud
aligned_ply = PlyElement.describe(aligned_data, 'vertex')

output_file = os.path.splitext(input_file)[0] + "_moi_aligned.ply"  # Add suffix to the file name
PlyData([aligned_ply]).write(output_file)
print(f"Aligned point cloud saved with attributes as: {output_file}")


Aligned point cloud saved with attributes as: /home/ubuntu/scene/EchoScene/assets/scenes/bodleian0025_moi_aligned.ply


In [15]:

def align_point_cloud(input_file, output_file):
    # Load the point cloud
    pcd = o3d.io.read_point_cloud(input_file)

    # Step 1: Detect floor plane using RANSAC
    plane_model, inliers = pcd.segment_plane(distance_threshold=0.01,
                                             ransac_n=3,
                                             num_iterations=1000)
    [a, b, c, d] = plane_model
    print(f"Floor plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

    # Extract the floor normal
    floor_normal = np.array([a, b, c])
    target_floor_normal = np.array([0, 1, 0])  # Y-axis for XZ alignment

    # Calculate rotation to align floor to XZ plane
    rotation_axis_floor = np.cross(floor_normal, target_floor_normal)
    rotation_angle_floor = np.arccos(
        np.dot(floor_normal, target_floor_normal) / (np.linalg.norm(floor_normal) * np.linalg.norm(target_floor_normal))
    )
    rotation_matrix_floor = o3d.geometry.get_rotation_matrix_from_axis_angle(rotation_axis_floor * rotation_angle_floor)

    # Apply rotation to align the floor
    pcd.rotate(rotation_matrix_floor, center=(0, 0, 0))

    # Step 2: Detect wall planes after floor alignment
    wall_plane_model, wall_inliers = pcd.segment_plane(distance_threshold=0.01,
                                                       ransac_n=3,
                                                       num_iterations=1000)
    [wa, wb, wc, wd] = wall_plane_model
    print(f"Wall plane equation: {wa:.2f}x + {wb:.2f}y + {wc:.2f}z + {wd:.2f} = 0")

    # Extract wall normal
    wall_normal = np.array([wa, wb, wc])

    # Determine target alignment for the wall
    if abs(wall_normal[0]) > abs(wall_normal[2]):
        target_wall_normal = np.array([1, 0, 0])  # Align to YZ plane
    else:
        target_wall_normal = np.array([0, 0, 1])  # Align to YX plane

    # Calculate rotation to align wall
    rotation_axis_wall = np.cross(wall_normal, target_wall_normal)
    rotation_angle_wall = np.arccos(
        np.dot(wall_normal, target_wall_normal) / (np.linalg.norm(wall_normal) * np.linalg.norm(target_wall_normal))
    )
    rotation_matrix_wall = o3d.geometry.get_rotation_matrix_from_axis_angle(rotation_axis_wall * rotation_angle_wall)

    # Apply rotation to align walls
    pcd.rotate(rotation_matrix_wall, center=(0, 0, 0))

    # Save the aligned point cloud with Gaussian splat attributes
    save_point_cloud_with_attributes(pcd, input_file, output_file)

def save_point_cloud_with_attributes(pcd, input_file, output_file):
    # Read the original PLY file to preserve attributes
    ply_data = PlyData.read(input_file)
    vertex_data = ply_data['vertex'].data

    # Extract aligned points and normals
    aligned_points = np.asarray(pcd.points)
    aligned_normals = np.asarray(pcd.normals) if pcd.has_normals() else np.zeros_like(aligned_points)

    # Extract additional attributes
    f_dc = np.column_stack([vertex_data[f'f_dc_{i}'] for i in range(3)]) if 'f_dc_0' in vertex_data.dtype.names else np.zeros((aligned_points.shape[0], 3))
    f_rest = np.column_stack([vertex_data[f'f_rest_{i}'] for i in range(3)]) if 'f_rest_0' in vertex_data.dtype.names else np.zeros((aligned_points.shape[0], 3))
    opacity = vertex_data['opacity'] if 'opacity' in vertex_data.dtype.names else np.ones((aligned_points.shape[0], 1))
    scale = np.column_stack([vertex_data[f'scale_{i}'] for i in range(3)]) if 'scale_0' in vertex_data.dtype.names else np.ones((aligned_points.shape[0], 3))
    rotation = np.column_stack([vertex_data[f'rot_{i}'] for i in range(4)]) if 'rot_0' in vertex_data.dtype.names else np.zeros((aligned_points.shape[0], 4))

    # Combine all attributes
    aligned_data = np.zeros(aligned_points.shape[0], dtype=[
        ('x', 'f4'), ('y', 'f4'), ('z', 'f4'),
        ('nx', 'f4'), ('ny', 'f4'), ('nz', 'f4'),
        ('f_dc_0', 'f4'), ('f_dc_1', 'f4'), ('f_dc_2', 'f4'),
        ('f_rest_0', 'f4'), ('f_rest_1', 'f4'), ('f_rest_2', 'f4'),
        ('opacity', 'f4'),
        ('scale_0', 'f4'), ('scale_1', 'f4'), ('scale_2', 'f4'),
        ('rot_0', 'f4'), ('rot_1', 'f4'), ('rot_2', 'f4'), ('rot_3', 'f4'),
    ])

    aligned_data['x'], aligned_data['y'], aligned_data['z'] = aligned_points[:, 0], aligned_points[:, 1], aligned_points[:, 2]
    aligned_data['nx'], aligned_data['ny'], aligned_data['nz'] = aligned_normals[:, 0], aligned_normals[:, 1], aligned_normals[:, 2]
    aligned_data['f_dc_0'], aligned_data['f_dc_1'], aligned_data['f_dc_2'] = f_dc[:, 0], f_dc[:, 1], f_dc[:, 2]
    aligned_data['f_rest_0'], aligned_data['f_rest_1'], aligned_data['f_rest_2'] = f_rest[:, 0], f_rest[:, 1], f_rest[:, 2]
    aligned_data['opacity'] = opacity
    aligned_data['scale_0'], aligned_data['scale_1'], aligned_data['scale_2'] = scale[:, 0], scale[:, 1], scale[:, 2]
    aligned_data['rot_0'], aligned_data['rot_1'], aligned_data['rot_2'], aligned_data['rot_3'] = rotation[:, 0], rotation[:, 1], rotation[:, 2], rotation[:, 3]

    # Save the aligned point cloud
    aligned_ply = PlyElement.describe(aligned_data, 'vertex')
    PlyData([aligned_ply]).write(output_file)

    print(f"Aligned point cloud with attributes saved as: {output_file}")

output_file = os.path.splitext(input_file)[0] + "_wall_and_floor.ply"  # Add suffix to the file name
align_point_cloud(input_file, output_file)


Floor plane equation: -0.00x + 0.99y + -0.16z + -0.53 = 0
Wall plane equation: -0.01x + 0.99y + -0.15z + -0.50 = 0
Aligned point cloud with attributes saved as: /home/ubuntu/scene/EchoScene/assets/scenes/bodleian0025_wall_and_floor.ply


In [16]:
import numpy as np
import open3d as o3d
from plyfile import PlyData, PlyElement
import os

def align_principal_axes(input_file, output_file):
    # Load the point cloud
    pcd = o3d.io.read_point_cloud(input_file)

    # Extract the points
    points = np.asarray(pcd.points)

    # Step 1: Compute the centroid of the point cloud
    centroid = np.mean(points, axis=0)

    # Center the point cloud at the origin
    centered_points = points - centroid

    # Step 2: Compute the covariance matrix
    covariance_matrix = np.cov(centered_points.T)

    # Step 3: Perform eigenvalue decomposition
    eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)

    # Sort eigenvectors by eigenvalues (largest to smallest)
    sorted_indices = np.argsort(eigenvalues)[::-1]
    eigenvectors = eigenvectors[:, sorted_indices]

    # Step 4: Create a rotation matrix to align principal axes to world axes
    # The eigenvectors already represent the axes alignment
    rotation_matrix = eigenvectors.T  # Transpose to match Open3D's convention

    # Apply rotation to the point cloud
    pcd.rotate(rotation_matrix, center=(0, 0, 0))

    # Step 5: Reassign normals if available
    if pcd.has_normals():
        normals = np.asarray(pcd.normals)
        rotated_normals = np.dot(normals, rotation_matrix.T)
        pcd.normals = o3d.utility.Vector3dVector(rotated_normals)

    # Save the aligned point cloud with preserved attributes
    save_point_cloud_with_attributes(pcd, input_file, output_file)

def save_point_cloud_with_attributes(pcd, input_file, output_file):
    # Read the original PLY file to preserve attributes
    ply_data = PlyData.read(input_file)
    vertex_data = ply_data['vertex'].data

    # Extract aligned points and normals
    aligned_points = np.asarray(pcd.points)
    aligned_normals = np.asarray(pcd.normals) if pcd.has_normals() else np.zeros_like(aligned_points)

    # Extract additional attributes
    f_dc = np.column_stack([vertex_data[f'f_dc_{i}'] for i in range(3)]) if 'f_dc_0' in vertex_data.dtype.names else np.zeros((aligned_points.shape[0], 3))
    f_rest = np.column_stack([vertex_data[f'f_rest_{i}'] for i in range(3)]) if 'f_rest_0' in vertex_data.dtype.names else np.zeros((aligned_points.shape[0], 3))
    opacity = vertex_data['opacity'] if 'opacity' in vertex_data.dtype.names else np.ones((aligned_points.shape[0], 1))
    scale = np.column_stack([vertex_data[f'scale_{i}'] for i in range(3)]) if 'scale_0' in vertex_data.dtype.names else np.ones((aligned_points.shape[0], 3))
    rotation = np.column_stack([vertex_data[f'rot_{i}'] for i in range(4)]) if 'rot_0' in vertex_data.dtype.names else np.zeros((aligned_points.shape[0], 4))

    # Combine all attributes
    aligned_data = np.zeros(aligned_points.shape[0], dtype=[
        ('x', 'f4'), ('y', 'f4'), ('z', 'f4'),
        ('nx', 'f4'), ('ny', 'f4'), ('nz', 'f4'),
        ('f_dc_0', 'f4'), ('f_dc_1', 'f4'), ('f_dc_2', 'f4'),
        ('f_rest_0', 'f4'), ('f_rest_1', 'f4'), ('f_rest_2', 'f4'),
        ('opacity', 'f4'),
        ('scale_0', 'f4'), ('scale_1', 'f4'), ('scale_2', 'f4'),
        ('rot_0', 'f4'), ('rot_1', 'f4'), ('rot_2', 'f4'), ('rot_3', 'f4'),
    ])

    aligned_data['x'], aligned_data['y'], aligned_data['z'] = aligned_points[:, 0], aligned_points[:, 1], aligned_points[:, 2]
    aligned_data['nx'], aligned_data['ny'], aligned_data['nz'] = aligned_normals[:, 0], aligned_normals[:, 1], aligned_normals[:, 2]
    aligned_data['f_dc_0'], aligned_data['f_dc_1'], aligned_data['f_dc_2'] = f_dc[:, 0], f_dc[:, 1], f_dc[:, 2]
    aligned_data['f_rest_0'], aligned_data['f_rest_1'], aligned_data['f_rest_2'] = f_rest[:, 0], f_rest[:, 1], f_rest[:, 2]
    aligned_data['opacity'] = opacity
    aligned_data['scale_0'], aligned_data['scale_1'], aligned_data['scale_2'] = scale[:, 0], scale[:, 1], scale[:, 2]
    aligned_data['rot_0'], aligned_data['rot_1'], aligned_data['rot_2'], aligned_data['rot_3'] = rotation[:, 0], rotation[:, 1], rotation[:, 2], rotation[:, 3]

    # Save the aligned point cloud
    aligned_ply = PlyElement.describe(aligned_data, 'vertex')
    PlyData([aligned_ply]).write(output_file)

    print(f"Aligned point cloud with attributes saved as: {output_file}")

# Run the alignment process
output_file = os.path.splitext(input_file)[0] + "_pca_aligned.ply"  # Add suffix to the file name
align_principal_axes(input_file, output_file)


Aligned point cloud with attributes saved as: /home/ubuntu/scene/EchoScene/assets/scenes/bodleian0025_pca_aligned.ply


In [ ]:
import numpy as np
import open3d as o3d
from plyfile import PlyData, PlyElement
import os

def align_to_bounding_box(input_file, output_file):
    # Load the point cloud
    pcd = o3d.io.read_point_cloud(input_file)

    # Step 1: Compute the Oriented Bounding Box (OBB)
    obb = pcd.get_oriented_bounding_box()

    # Step 2: Extract the rotation matrix of the OBB
    obb_rotation_matrix = obb.R
    print("OBB Rotation Matrix:")
    print(obb_rotation_matrix)

    # Step 3: Compute the inverse of the OBB rotation matrix to align to XYZ axes
    inverse_rotation_matrix = np.linalg.inv(obb_rotation_matrix)

    # Step 4: Apply the transformation to the point cloud
    pcd.rotate(inverse_rotation_matrix, center=obb.center)

    # Reassign normals if available
    if pcd.has_normals():
        normals = np.asarray(pcd.normals)
        rotated_normals = np.dot(normals, inverse_rotation_matrix.T)
        pcd.normals = o3d.utility.Vector3dVector(rotated_normals)

    # Save the aligned point cloud with preserved attributes
    save_point_cloud_with_attributes(pcd, input_file, output_file)

def save_point_cloud_with_attributes(pcd, input_file, output_file):
    # Read the original PLY file to preserve attributes
    ply_data = PlyData.read(input_file)
    vertex_data = ply_data['vertex'].data

    # Extract aligned points and normals
    aligned_points = np.asarray(pcd.points)
    aligned_normals = np.asarray(pcd.normals) if pcd.has_normals() else np.zeros_like(aligned_points)

    # Extract additional attributes
    f_dc = np.column_stack([vertex_data[f'f_dc_{i}'] for i in range(3)]) if 'f_dc_0' in vertex_data.dtype.names else np.zeros((aligned_points.shape[0], 3))
    f_rest = np.column_stack([vertex_data[f'f_rest_{i}'] for i in range(3)]) if 'f_rest_0' in vertex_data.dtype.names else np.zeros((aligned_points.shape[0], 3))
    opacity = vertex_data['opacity'] if 'opacity' in vertex_data.dtype.names else np.ones((aligned_points.shape[0], 1))
    scale = np.column_stack([vertex_data[f'scale_{i}'] for i in range(3)]) if 'scale_0' in vertex_data.dtype.names else np.ones((aligned_points.shape[0], 3))
    rotation = np.column_stack([vertex_data[f'rot_{i}'] for i in range(4)]) if 'rot_0' in vertex_data.dtype.names else np.zeros((aligned_points.shape[0], 4))

    # Combine all attributes
    aligned_data = np.zeros(aligned_points.shape[0], dtype=[
        ('x', 'f4'), ('y', 'f4'), ('z', 'f4'),
        ('nx', 'f4'), ('ny', 'f4'), ('nz', 'f4'),
        ('f_dc_0', 'f4'), ('f_dc_1', 'f4'), ('f_dc_2', 'f4'),
        ('f_rest_0', 'f4'), ('f_rest_1', 'f4'), ('f_rest_2', 'f4'),
        ('opacity', 'f4'),
        ('scale_0', 'f4'), ('scale_1', 'f4'), ('scale_2', 'f4'),
        ('rot_0', 'f4'), ('rot_1', 'f4'), ('rot_2', 'f4'), ('rot_3', 'f4'),
    ])

    aligned_data['x'], aligned_data['y'], aligned_data['z'] = aligned_points[:, 0], aligned_points[:, 1], aligned_points[:, 2]
    aligned_data['nx'], aligned_data['ny'], aligned_data['nz'] = aligned_normals[:, 0], aligned_normals[:, 1], aligned_normals[:, 2]
    aligned_data['f_dc_0'], aligned_data['f_dc_1'], aligned_data['f_dc_2'] = f_dc[:, 0], f_dc[:, 1], f_dc[:, 2]
    aligned_data['f_rest_0'], aligned_data['f_rest_1'], aligned_data['f_rest_2'] = f_rest[:, 0], f_rest[:, 1], f_rest[:, 2]
    aligned_data['opacity'] = opacity
    aligned_data['scale_0'], aligned_data['scale_1'], aligned_data['scale_2'] = scale[:, 0], scale[:, 1], scale[:, 2]
    aligned_data['rot_0'], aligned_data['rot_1'], aligned_data['rot_2'], aligned_data['rot_3'] = rotation[:, 0], rotation[:, 1], rotation[:, 2], rotation[:, 3]

    # Save the aligned point cloud
    aligned_ply = PlyElement.describe(aligned_data, 'vertex')
    PlyData([aligned_ply]).write(output_file)

    print(f"Aligned point cloud with attributes saved as: {output_file}")

output_file = os.path.splitext(input_file)[0] + "_obb_aligned.ply"  # Add suffix to the file name
align_to_bounding_box(input_file, output_file)
